<a href="https://colab.research.google.com/github/Pawan-KS/ML-Library-1/blob/master/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
%matplotlib inline

In [23]:
d= pd.read_csv('sample_data/mnist_train_small.csv',header=None)
data=np.array(d)
print(data)

[[6 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]
 [7 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [9 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]


In [0]:
def convert_data(data,concerned_class,threshold=5):
  data[:,1:][data[:,1:]>=threshold]=1
  data[:,0][data[:,0]!=concerned_class]=-1
  data[:,0][data[:,0]==concerned_class]=0
  data[:,0]+=1
  return data

In [0]:
class decision_tree():
  def __init__(self,data):
    self.comp_data=data
    self.i=1

  def calc_gini(self,data):
    if data.size==0:
      return 0
    d=data[:,0]
    p = np.count_nonzero(d==1)/len(d)
    return (p*p)+((1-p)*(1-p))

  def gini_split_val(self,data,col,split='False'):
    d=data[:,col]
    d1=[]
    d2=[]
    for i in range(data.shape[0]):
      if d[i]==0:
        d1.append(data[i,:])
      elif d[i]==1:
        d2.append(data[i,:])
    d1=np.array(d1)
    d2=np.array(d2)
    gini1=self.calc_gini(d1)
    gini2=self.calc_gini(d2)
    if split=='True':
      if d1.size==0:
        return d1,np.delete(d2,[col],1)
      elif d2.size==0:
        return np.delete(d1,[col],1),d2
      else:
        return np.delete(d1,[col],1),np.delete(d2,[col],1)
    if gini1==0:
      return gini2
    elif gini2==0:
      return gini1
    #check if both gini are not 0
    else :
      return ((d1.shape[0]/len(d))*gini1) + ((d2.shape[0]/len(d))*gini2)
    
  def choosing_col(self,data):
    gini_vals=[]
    for i in range(data.shape[1]-1):
      gini_vals.append(self.gini_split_val(data,i+1))
    return np.argmax(np.array(gini_vals))+1
  
  def build_tree(self,dataset,parent_class='None'):
    #print(self.i)
    self.i+=1
    #print(dataset)
    if dataset.size==0:
      return np.unique(self.comp_data[:,0])[np.argmax(np.unique(self.comp_data[:,0],return_counts=True)[1])]
    elif len(np.unique(dataset[:,0]))<=1:
      return dataset[0][0]
    elif dataset.shape==(dataset.shape[0],1):
      return parent_class
    else:
      parent_class= np.unique(dataset[:,0])[np.argmax(np.unique(dataset[:,0],return_counts=True)[1])]
      attr_no = self.choosing_col(dataset)
      tree = {attr_no:{}}
      #for value in np.unique(dataset[attr_no]):
      for value in range(2):
        t = self.gini_split_val(dataset,attr_no,split='True')
        subtree= self.build_tree(t[value],parent_class)
        tree[attr_no][value]=subtree
      return tree
  
  def prediction(self,query,tree,default_val='None'):
    if default_val=='None':
      default_val=np.unique(self.comp_data[:,0])[np.argmax(np.unique(self.comp_data[:,0],return_counts=True)[1])]
    for i in list(query.keys()):
      if i in list(tree.keys()):
        try:
          out= tree[i][query[i]]
        except:
          return default_val
        out = tree[i][query[i]]
        if isinstance(out,dict):
          return self.prediction(query,out)
        else:
          return out
  
  def test(self,data,tree):
    test_data=data[:,1:]
    y=data[:,0]
    count=0
    for i in range(data.shape[0]):
      test_query=dict(enumerate(test_data[i,:]))
      pred=self.prediction(test_query,tree)
      if pred==y[i]:
        count+=1
    print((count/len(y))*100)



data= convert_data(data,0,10)
obj=decision_tree(data[:100,:])
tree= obj.build_tree(data[:100,:])    

In [25]:
obj.test(data,tree)

94.22500000000001


In [30]:
d_t= pd.read_csv('sample_data/mnist_test.csv',header=None)
data_t=np.array(d_t)

data_t=convert_data(data_t,0,10)
obj.test(data_t,tree)

94.75


In [28]:
print(tree)

{436: {0: {353: {0: {440: {0: {326: {0: 0, 1: 1}}, 1: 1}}, 1: 0}}, 1: 0}}
